<div style="text-align: left;">
<table style="width:100%; background-color:transparent;">
  <tr style="background-color:transparent;">
    <td style="background-color:transparent;"><img href="http://www.datascience-paris-saclay.fr" src="http://project.inria.fr/saclaycds/files/2017/02/logoUPSayPlusCDS_990.png" ></td>
    <td style="background-color:transparent;"><img href="https://research.pasteur.fr/en/team/group-roberto-toro" src="https://paris-saclay-cds.github.io/autism_challenge/images/institut_pasteur_logo.svg" ></td>
    <td style="background-color:black;"><img href="fer.unizg.hr" src="https://www.fer.unizg.hr/_pub/themes_static/fer2016/default/img/FER_logo.png"></td>
  </tr>
</table> 
</div>

<center><h1>Impact of sMRI preprocessing on autism classification using machine learning methods</h1></center>



<center><h3>Forked from data challenge on Autism Spectrum Disorder detection</h3></center>
<br/>
<center>_Roberto Toro (Institut Pasteur), Nicolas Traut (Institut Pasteur), Anita Beggiato (Institut Pasteur), Katja Heuer (Institut Pasteur),<br /> Gael Varoquaux (Inria, Parietal), Alex Gramfort (Inria, Parietal), Balazs Kegl (LAL),<br /> Guillaume Lemaitre (CDS), Alexandre Boucaud (CDS), and Joris van den Bossche (CDS)<br />Lana Barić(FER), Roko Krstičević(FER)</center>

## Table of Content

0. [Prerequisites](#Software-prerequisites)
1. [Introduction about the competition](#Introduction:-what-is-this-challenge-about)
3. [The data](#The-data)
4. [Workflow](#Workflow)
5. [Evaluation](#Evaluation)
6. [Submission](#Submitting-to-the-online-challenge:-ramp.studio)
7. [More information](#More-information)
8. [Questions](#Question)

**To download and run this notebook**: download the [full starting kit](https://github.com/ramp-kits/autism/archive/master.zip), with all the necessary files.

## Software prerequisites

This starting kit requires the following dependencies:

* `numpy`
* `scipy`
* `pandas`
* `scikit-learn`
* `matplolib`
* `seaborn`
* `nilearn`
* `jupyter`
* `ramp-workflow`

The following 2 cells will install if necessary the missing dependencies.

In [ ]:
# import sys
# !{sys.executable} -m pip install scikit-learn==0.21.3 seaborn==0.10.0 nilearn==0.7.1

  Using cached scikit-learn-0.21.3.tar.gz (12.2 MB)
  Preparing metadata (setup.py) ... done
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.20.4
    Uninstalling scikit-learn-0.20.4:
      Successfully uninstalled scikit-learn-0.20.4
  DEPRECATION: scikit-learn is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for scikit-learn ... /

Install `ramp-workflow` from the master branch on GitHub.

In [ ]:
# !{sys.executable} -m pip install https://api.github.com/repos/paris-saclay-cds/ramp-workflow/zipball/0.2.1

In [3]:
%load_ext autoreload
%autoreload 2

## Introduction: detecting autism

Autism spectrum disorder (ASD) is a developmental disorder affecting communication and behavior with different range in severity of symptoms. ASD has been reported to affect approximately 1 in 166 children.

Although there is a consensus on a relation between ASD and atypical brain networks and anatomy, those differences in brain anatomy and functional connectivity remain unclear. To address these issues, study on large cohort of subjects are necessary to ensure relevant finding. 

## The data
We start from downloading data from internet

In [47]:
from problem import get_train_data

data_train = get_train_data()

In [48]:
data_train.head(10)

,participants_site,participants_sex,participants_age,participants_asd,anatomy_lh_bankssts_area,anatomy_lh_caudalanteriorcingulate_area,anatomy_lh_caudalmiddlefrontal_area,anatomy_lh_cuneus_area,anatomy_lh_entorhinal_area,anatomy_lh_fusiform_area,...,fmri_basc064,fmri_basc122,fmri_basc197,fmri_craddock_scorr_mean,fmri_harvard_oxford_cort_prob_2mm,fmri_motions,fmri_msdl,fmri_power_2011,fmri_select,repetition_time
subject_id,,,,,,,,,,,,,,,,,,,,,
1932355398536124106,5,F,9.301370,0,977.0,427.0,1884.0,1449.0,463.0,2790.0,...,./data/fmri/basc064/1932355398536124106/run_1/...,./data/fmri/basc122/1932355398536124106/run_1/...,./data/fmri/basc197/1932355398536124106/run_1/...,./data/fmri/craddock_scorr_mean/19323553985361...,./data/fmri/harvard_oxford_cort_prob_2mm/19323...,./data/fmri/motions/1932355398536124106/run_1/...,./data/fmri/msdl/1932355398536124106/run_1/193...,./data/fmri/power_2011/1932355398536124106/run...,1,2.5
5174041730092253771,19,M,29.000000,0,1279.0,730.0,2419.0,1611.0,467.0,3562.0,...,./data/fmri/basc064/5174041730092253771/run_1/...,./data/fmri/basc122/5174041730092253771/run_1/...,./data/fmri/basc197/5174041730092253771/run_1/...,./data/fmri/craddock_scorr_mean/51740417300922...,./data/fmri/harvard_oxford_cort_prob_2mm/51740...,./data/fmri/motions/5174041730092253771/run_1/...,./data/fmri/msdl/5174041730092253771/run_1/517...,./data/fmri/power_2011/5174041730092253771/run...,1,3.0
10219322676643534800,19,F,45.000000,1,926.0,446.0,1897.0,2135.0,570.0,3064.0,...,./data/fmri/basc064/10219322676643534800/run_1...,./data/fmri/basc122/10219322676643534800/run_1...,./data/fmri/basc197/10219322676643534800/run_1...,./data/fmri/craddock_scorr_mean/10219322676643...,./data/fmri/harvard_oxford_cort_prob_2mm/10219...,./data/fmri/motions/10219322676643534800/run_1...,./data/fmri/msdl/10219322676643534800/run_1/10...,./data/fmri/power_2011/10219322676643534800/ru...,1,3.0
10645466564919190227,5,F,9.216438,1,983.0,588.0,2479.0,1312.0,525.0,3766.0,...,./data/fmri/basc064/10645466564919190227/run_1...,./data/fmri/basc122/10645466564919190227/run_1...,./data/fmri/basc197/10645466564919190227/run_1...,./data/fmri/craddock_scorr_mean/10645466564919...,./data/fmri/harvard_oxford_cort_prob_2mm/10645...,./data/fmri/motions/10645466564919190227/run_1...,./data/fmri/msdl/10645466564919190227/run_1/10...,./data/fmri/power_2011/10645466564919190227/ru...,1,2.5
14512541342641936232,28,M,15.050000,1,1488.0,593.0,2309.0,1829.0,726.0,3720.0,...,./data/fmri/basc064/14512541342641936232/run_1...,./data/fmri/basc122/14512541342641936232/run_1...,./data/fmri/basc197/14512541342641936232/run_1...,./data/fmri/craddock_scorr_mean/14512541342641...,./data/fmri/harvard_oxford_cort_prob_2mm/14512...,./data/fmri/motions/14512541342641936232/run_1...,./data/fmri/msdl/14512541342641936232/run_1/14...,./data/fmri/power_2011/14512541342641936232/ru...,1,1.0
7424332414628877016,0,M,22.000000,1,1138.0,581.0,2093.0,1573.0,642.0,3016.0,...,./data/fmri/basc064/7424332414628877016/run_1/...,./data/fmri/basc122/7424332414628877016/run_1/...,./data/fmri/basc197/7424332414628877016/run_1/...,./data/fmri/craddock_scorr_mean/74243324146288...,./data/fmri/harvard_oxford_cort_prob_2mm/74243...,./data/fmri/motions/7424332414628877016/run_1/...,./data/fmri/msdl/7424332414628877016/run_1/742...,./data/fmri/power_2011/7424332414628877016/run...,1,3.0
4084002381481689753,16,M,10.840000,0,1068.0,611.0,2234.0,1598.0,617.0,4008.0,...,./data/fmri/basc064/4084002381481689753/run_1/...,./data/fmri/basc122/4084002381481689753/run_1/...,./data/fmri/basc197/4084002381481689753/run_1/...,./data/fmri/craddock_scorr_mean/40840023814816...,./data/fmri/harvard_oxford_cort_prob_2mm/40840...,./data/fmri/motions/4084002381481689753/run_1/...,./data/fmri/msdl/4084002381481689753/run_1/408...,./data/fmri/power_2011/4084002381481689753/run...,1,2.5
3573738674348564622,5,M,9.471233,0,973.0,543.0,2152.0,1083.0,569.0,3390.0,...,./data/fmri/basc064/3573738674348564622/run_1/...,./data/fmri/basc122/3

In [28]:
print(data_train['participants_asd'])

subject_id
1932355398536124106     0
5174041730092253771     0
10219322676643534800    1
10645466564919190227    1
14512541342641936232    1
                       ..
14661172306020020468    0
1061335839335536120     1
7789126193351344754     0
9057622900394015793     1
10099127552657342413    0
Name: participants_asd, Length: 1127, dtype: int64


In [19]:
print('Number of subjects in the training tests: {}'.format(data_train['participants_asd'].size))

Number of subjects in the training tests: 1127


#### Participant features

In [4]:
data_train_participants = data_train[[col for col in data_train.columns if col.startswith('participants')]]
data_train_participants.head()

,participants_site,participants_sex,participants_age,participants_asd
subject_id,,,,
1932355398536124106,5,F,9.301370,0
5174041730092253771,19,M,29.000000,0
10219322676643534800,19,F,45.000000,1
10645466564919190227,5,F,9.216438,1
14512541342641936232,28,M,15.050000,1


#### Structural MRI features

A set of structural features have been extracted for each subject: (i) normalized brain volume computed using subcortical segmentation of FreeSurfer and (ii) cortical thickness and area for right and left hemisphere of FreeSurfer.

In [5]:
data_train_anatomy = data_train[[col for col in data_train.columns if col.startswith('anatomy')]]
data_train_anatomy.head()

,anatomy_lh_bankssts_area,anatomy_lh_caudalanteriorcingulate_area,anatomy_lh_caudalmiddlefrontal_area,anatomy_lh_cuneus_area,anatomy_lh_entorhinal_area,anatomy_lh_fusiform_area,anatomy_lh_inferiorparietal_area,anatomy_lh_inferiortemporal_area,anatomy_lh_isthmuscingulate_area,anatomy_lh_lateraloccipital_area,...,anatomy_MaskVol,anatomy_BrainSegVol-to-eTIV,anatomy_MaskVol-to-eTIV,anatomy_lhSurfaceHoles,anatomy_rhSurfaceHoles,anatomy_SurfaceHoles,anatomy_EstimatedTotalIntraCranialVol,anatomy_eTIV,anatomy_BrainSegVolNotVent,anatomy_select
subject_id,,,,,,,,,,,,,,,,,,,,,
1932355398536124106,977.0,427.0,1884.0,1449.0,463.0,2790.0,4091.0,3305.0,897.0,4406.0,...,1375171.0,0.840976,1.077472,30.0,31.0,61.0,1.276294e+06,1.276294e+06,1058903.0,1
5174041730092253771,1279.0,730.0,2419.0,1611.0,467.0,3562.0,5380.0,3555.0,1155.0,5611.0,...,1807924.0,0.771229,1.033285,45.0,54.0,99.0,1.749685e+06,1.749685e+06,1329340.0,1
10219322676643534800,926.0,446.0,1897.0,2135.0,570.0,3064.0,4834.0,2602.0,1171.0,6395.0,...,1522076.0,0.774117,1.082107,48.0,50.0,98.0,1.406585e+06,1.406585e+06,1072503.0,1
10645466564919190227,983.0,588.0,2479.0,1312.0,525.0,3766.0,5091.0,3433.0,1028.0,5405.0,...,1544951.0,0.845986,1.083437,55.0,57.0,112.0,1.425972e+06,1.425972e+06,1194831.0,1
14512541342641936232,1488.0,593.0,2309.0,1829.0,726.0,3720.0,5432.0,3956.0,1033.0,5644.0,...,1738955.0,0.793794,1.083640,22.0,42.0,64.0,1.604735e+06,1.604735e+06,1263065.0,1


Note that the column `anatomy_select` contain a label affected during a manual quality check (i.e. `0` and `3` reject, `1` accept, `2` accept with reserve). This column can be used during training to exclude noisy data for instance.

In [6]:
data_train_anatomy['anatomy_select'].head()

subject_id
1932355398536124106     1
5174041730092253771     1
10219322676643534800    1
10645466564919190227    1
14512541342641936232    1
Name: anatomy_select, dtype: int64

#### Testing data

The testing data can be loaded similarly as follows:

In [25]:
from problem import get_test_data

data_test, labels_test = get_test_data()

In [26]:
data_test.head()

,participants_site,participants_sex,participants_age,anatomy_lh_bankssts_area,anatomy_lh_caudalanteriorcingulate_area,anatomy_lh_caudalmiddlefrontal_area,anatomy_lh_cuneus_area,anatomy_lh_entorhinal_area,anatomy_lh_fusiform_area,anatomy_lh_inferiorparietal_area,...,fmri_basc064,fmri_basc122,fmri_basc197,fmri_craddock_scorr_mean,fmri_harvard_oxford_cort_prob_2mm,fmri_motions,fmri_msdl,fmri_power_2011,fmri_select,repetition_time
subject_id,,,,,,,,,,,,,,,,,,,,,
5181409268393785348,31,M,12.200000,985.0,723.0,2851.0,1844.0,495.0,3526.0,5658.0,...,./data/fmri/basc064/5181409268393785348/run_1/...,./data/fmri/basc122/5181409268393785348/run_1/...,./data/fmri/basc197/5181409268393785348/run_1/...,./data/fmri/craddock_scorr_mean/51814092683937...,./data/fmri/harvard_oxford_cort_prob_2mm/51814...,./data/fmri/motions/5181409268393785348/run_1/...,./data/fmri/msdl/5181409268393785348/run_1/518...,./data/fmri/power_2011/5181409268393785348/run...,1,2.0
8797865049371315550,9,M,14.000000,1174.0,506.0,1890.0,1327.0,462.0,3564.0,4408.0,...,./data/fmri/basc064/8797865049371315550/run_1/...,./data/fmri/basc122/8797865049371315550/run_1/...,./data/fmri/basc197/8797865049371315550/run_1/...,./data/fmri/craddock_scorr_mean/87978650493713...,./data/fmri/harvard_oxford_cort_prob_2mm/87978...,./data/fmri/motions/8797865049371315550/run_1/...,./data/fmri/msdl/8797865049371315550/run_1/879...,./data/fmri/power_2011/8797865049371315550/run...,1,2.5
6486385878325245147,20,M,14.425000,1288.0,568.0,2406.0,1546.0,432.0,3497.0,4808.0,...,./data/fmri/basc064/6486385878325245147/run_1/...,./data/fmri/basc122/6486385878325245147/run_1/...,./data/fmri/basc197/6486385878325245147/run_1/...,./data/fmri/craddock_scorr_mean/64863858783252...,./data/fmri/harvard_oxford_cort_prob_2mm/64863...,./data/fmri/motions/6486385878325245147/run_1/...,./data/fmri/msdl/6486385878325245147/run_1/648...,./data/fmri/power_2011/6486385878325245147/run...,1,2.0
17126438435398394588,33,M,22.880200,1179.0,991.0,2427.0,1771.0,363.0,3579.0,6082.0,...,./data/fmri/basc064/17126438435398394588/run_1...,./data/fmri/basc122/17126438435398394588/run_1...,./data/fmri/basc197/17126438435398394588/run_1...,./data/fmri/craddock_scorr_mean/17126438435398...,./data/fmri/harvard_oxford_cort_prob_2mm/17126...,./data/fmri/motions/17126438435398394588/run_1...,./data/fmri/msdl/17126438435398394588/run_1/17...,./data/fmri/power_2011/17126438435398394588/ru...,1,2.0
16638049522113999228,2,M,8.252055,1064.0,721.0,2445.0,1453.0,561.0,3262.0,4885.0,...,./data/fmri/basc064/16638049522113999228/run_1...,./data/fmri/basc122/16638049522113999228/run_1...,./data/fmri/basc197/16638049522113999228/run_1...,./data/fmri/craddock_scorr_mean/16638049522113...,./data/fmri/harvard_oxford_cort_prob_2mm/16638...,./data/fmri/motions/16638049522113999228/run_1...,./data/fmri/msdl/16638049522113999228/run_1/16...,./data/fmri/power_2011/16638049522113999228/ru...,0,2.0


In [27]:
print(labels_test)

[0 1 0 1 1 0 1 1 1 1 1 0 0 1 0 0 0 1 0 1 1 0 0]


## Workflow

<img src="./img/workflow2.png" width="100%">

### Quality selector

Quality selector works by chosing some amount of bad quality data from the data set based on input of quality that ranges from 0 to 1

In [50]:
import pandas as pd

def quality_selector(data, quality_factor = 1):

    bad_quality = 1 - min(2 * quality_factor, 1)
    decent_quality = quality_factor * 2
    if quality_factor > 0.5:
        decent_quality = 2 - 2 * quality_factor
    good_quality = 0
    if quality_factor > 0.5:
        good_quality = quality_factor * 2 - 1

    # First we get all the data with anatomy_select = 0 or 3
    bad_data = data[data['anatomy_select'].isin([0, 3])]

    # Then we get all the data with anatomy_select = 1
    decent_data = data[data['anatomy_select'] == 1]

    # Then we get all the data with anatomy_select = 2
    good_data = data[data['anatomy_select'] == 2]

    # Now we select the amount of data according to quality_factor
    if bad_quality > 0:
        bad_data = bad_data.sample(frac=bad_quality, replace=True)
    if decent_quality > 0:
        decent_data = decent_data.sample(frac=decent_quality, replace=True)
    if good_quality > 0:
        good_data = good_data.sample(frac=good_quality, replace=True)

    # Finally we concatenate the data
    dataframes = [df for df in [bad_data, decent_data, good_data] if not df.empty]
    if not dataframes:
        raise ValueError('All sampled data is empty. Try a larger quality_factor.')
    tmp_data = pd.concat(dataframes)

    return tmp_data


quality_selector(data_train, 0.6)


ValueError: Sampled data is empty. Try a larger quality_factor.

### Evaluation

The framework is evaluated with a cross-validation approach. The metrics used are the AUC under the ROC and the accuracy.

In [33]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from problem import get_cv

def evaluation(X, y):
    pipe = make_pipeline(FeatureExtractor(), Classifier())
    cv = get_cv(X, y)
    results = cross_validate(pipe, X, y, scoring=['roc_auc', 'accuracy'], cv=cv,
                             verbose=1, return_train_score=True,
                             n_jobs=1)
    
    return results

#### FeatureExtractor

The available structural data can be used directly to make some classification. In this regard, we will use a feature extractor (i.e. `FeatureExtractor`). This extractor will only select only the anatomical features, dropping any information regarding the fMRI-based features.

In [34]:
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin


class FeatureExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y):
        return self

    def transform(self, X_df):
        # get only the anatomical information
        X = X_df[[col for col in X_df.columns if col.startswith('anatomy')]]
        return X.drop(columns='anatomy_select')


#### Classifier

We propose to use a logistic classifier preceded from a scaler which will remove the mean and standard deviation computed on the training set.

In [35]:
from sklearn.base import BaseEstimator
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline


class Classifier(BaseEstimator):
    def __init__(self):
        self.clf = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear'))

    def fit(self, X, y):
        self.clf.fit(X, y)
        return self
        
    def predict(self, X):
        return self.clf.predict(X)

    def predict_proba(self, X):
        return self.clf.predict_proba(X)


#### Testing the submission

We can test locally our pipeline using `evaluation` function that we defined earlier.

In [36]:
import numpy as np

In [41]:
data_train = quality_selector(data_train, 0.1)

labels_train = data_train['participants_asd']

results = evaluation(data_train.drop('participants_asd', axis=1), labels_train)

print("Training score ROC-AUC: {:.3f} +- {:.3f}".format(np.mean(results['train_roc_auc']),
                                                        np.std(results['train_roc_auc'])))
print("Validation score ROC-AUC: {:.3f} +- {:.3f} \n".format(np.mean(results['test_roc_auc']),
                                                          np.std(results['test_roc_auc'])))

print("Training score accuracy: {:.3f} +- {:.3f}".format(np.mean(results['train_accuracy']),
                                                         np.std(results['train_accuracy'])))
print("Validation score accuracy: {:.3f} +- {:.3f}".format(np.mean(results['test_accuracy']),
                                                           np.std(results['test_accuracy'])))

ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

## More information

You can find more information in the [README](https://github.com/paris-saclay-cds/ramp-workflow/blob/master/README.md) of the [ramp-workflow library](https://github.com/paris-saclay-cds/ramp-workflow).